In [1]:
import torch
import numpy as np
from torch.nn.utils.rnn import pad_sequence, pack_sequence, pack_padded_sequence, pad_packed_sequence

In [2]:
# Random word from random word generator
data = ['hello world',
        'midnight',
        'calculation',
        'path',
        'short circuit']

# Make dictionary
char_set = ['<pad>'] + list(set(char for seq in data for char in seq)) # Get all characters and include pad token
char2idx = {char: idx for idx, char in enumerate(char_set)} # Constuct character to index dictionary
print('char_set:', char_set)
print('char_set length:', len(char_set))

char_set: ['<pad>', 'n', 'e', 'd', 'l', 'u', 't', ' ', 'a', 'r', 's', 'c', 'h', 'i', 'w', 'm', 'g', 'p', 'o']
char_set length: 19


In [3]:
# Convert character to index and make list of tensors
X = [torch.LongTensor([char2idx[char] for char in seq]) for seq in data]

# Check converted result
for sequence in X:
    print(sequence)

tensor([12,  2,  4,  4, 18,  7, 14, 18,  9,  4,  3])
tensor([15, 13,  3,  1, 13, 16, 12,  6])
tensor([11,  8,  4, 11,  5,  4,  8,  6, 13, 18,  1])
tensor([17,  8,  6, 12])
tensor([10, 12, 18,  9,  6,  7, 11, 13,  9, 11,  5, 13,  6])


In [4]:
# Make length tensor (will be used later in 'pack_padded_sequence' function)
lengths = [len(seq) for seq in X]
print('lengths:', lengths)

lengths: [11, 8, 11, 4, 13]


In [5]:
# Make a Tensor of shape (Batch x Maximum_Sequence_Length)
padded_sequence = pad_sequence(X, batch_first=True) # X is now padded sequence
print(padded_sequence)
print(padded_sequence.shape)

tensor([[12,  2,  4,  4, 18,  7, 14, 18,  9,  4,  3,  0,  0],
        [15, 13,  3,  1, 13, 16, 12,  6,  0,  0,  0,  0,  0],
        [11,  8,  4, 11,  5,  4,  8,  6, 13, 18,  1,  0,  0],
        [17,  8,  6, 12,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [10, 12, 18,  9,  6,  7, 11, 13,  9, 11,  5, 13,  6]])
torch.Size([5, 13])


In [6]:
# Sort by descending lengths
sorted_idx = sorted(range(len(lengths)), key=lengths.__getitem__, reverse=True)
sorted_X = [X[idx] for idx in sorted_idx]

# Check converted result
for sequence in sorted_X:
    print(sequence)

tensor([10, 12, 18,  9,  6,  7, 11, 13,  9, 11,  5, 13,  6])
tensor([12,  2,  4,  4, 18,  7, 14, 18,  9,  4,  3])
tensor([11,  8,  4, 11,  5,  4,  8,  6, 13, 18,  1])
tensor([15, 13,  3,  1, 13, 16, 12,  6])
tensor([17,  8,  6, 12])


In [7]:
packed_sequence = pack_sequence(sorted_X)
print(packed_sequence)

PackedSequence(data=tensor([10, 12, 11, 15, 17, 12,  2,  8, 13,  8, 18,  4,  4,  3,  6,  9,  4, 11,
         1, 12,  6, 18,  5, 13,  7,  7,  4, 16, 11, 14,  8, 12, 13, 18,  6,  6,
         9,  9, 13, 11,  4, 18,  5,  3,  1, 13,  6]), batch_sizes=tensor([5, 5, 5, 5, 4, 4, 4, 4, 3, 3, 3, 1, 1]), sorted_indices=None, unsorted_indices=None)


In [8]:
# one-hot embedding using PaddedSequence
eye = torch.eye(len(char_set)) # Identity matrix of shape (len(char_set), len(char_set))
embedded_tensor = eye[padded_sequence]
print(embedded_tensor.shape) # shape: (Batch_size, max_sequence_length, number_of_input_tokens)

torch.Size([5, 13, 19])


In [9]:
# one-hot embedding using PackedSequence
embedded_packed_seq = pack_sequence([eye[X[idx]] for idx in sorted_idx])
print(embedded_packed_seq.data.shape)

torch.Size([47, 19])


In [10]:
# declare RNN
rnn = torch.nn.RNN(input_size=len(char_set), hidden_size=30, batch_first=True)

In [11]:
rnn_output, hidden = rnn(embedded_tensor)
print(rnn_output.shape) # shape: (batch_size, max_seq_length, hidden_size)
print(hidden.shape)     # shape: (num_layers * num_directions, batch_size, hidden_size)

torch.Size([5, 13, 30])
torch.Size([1, 5, 30])


In [12]:
rnn_output, hidden = rnn(embedded_packed_seq)
print(rnn_output.data.shape)
print(hidden.data.shape)

torch.Size([47, 30])
torch.Size([1, 5, 30])


In [13]:
unpacked_sequence, seq_lengths = pad_packed_sequence(embedded_packed_seq, batch_first=True)
print(unpacked_sequence.shape)
print(seq_lengths)

torch.Size([5, 13, 19])
tensor([13, 11, 11,  8,  4])


In [14]:
embedded_padded_sequence = eye[pad_sequence(sorted_X, batch_first=True)]
print(embedded_padded_sequence.shape)

torch.Size([5, 13, 19])


In [15]:
sorted_lengths = sorted(lengths, reverse=True)
new_packed_sequence = pack_padded_sequence(embedded_padded_sequence, sorted_lengths, batch_first=True)
print(new_packed_sequence.data.shape)
print(new_packed_sequence.batch_sizes)

torch.Size([47, 19])
tensor([5, 5, 5, 5, 4, 4, 4, 4, 3, 3, 3, 1, 1])
